In [3]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split

def prepare_dataset(X, train = False, mean = 0, std =1):
    X["NameLength"] = X["Name"].apply(lambda x: len(x))

    X = X.drop(["Survived", "PassengerId", "Name", "Embarked", "Ticket", "Cabin", "SibSp"], axis=1, errors="ignore")

    X["Sex"] = X["Sex"].apply(lambda x: 1 if x == "male" else 0)

    for col in X.columns:
        X[col] = X[col].fillna(X[col].mean())
    
    X["Child"] = X["Age"].apply(lambda x: 1 if x < 15 else 0)

    X["Expensive"] = X["Fare"].apply(lambda x: 1 if x < 18 else 0)

    if train:
        mean = X.mean()
        std = X.std()
        X = (X - mean)/std
    else:
        X = (X - mean)/std

    X = X.drop(["Age", "Fare"], axis = 1)

    if train:
        return X, mean, std
    return X


def load_datasets():
    x_train = pd.read_csv(os.path.join("titanic_data", "train.csv"))
    x_test = pd.read_csv(os.path.join("titanic_data", "test.csv"))
    
    x_train, x_val = train_test_split(x_train, test_size=0.1)
    
    y_train = x_train["Survived"]
    y_val = x_val["Survived"]
    y_test = pd.read_csv(os.path.join("titanic_data", "gender_submission.csv"))["Survived"]
    
    x_train, mean, std = prepare_dataset(x_train, train=True)
    x_val = prepare_dataset(x_val, mean=mean, std=std)
    x_test = prepare_dataset(x_test, mean=mean, std=std)

    return x_train, y_train, x_val, y_val, x_test, y_test

x_train, y_train, x_val, y_val, x_test, y_test = load_datasets()


Index(['Pclass', 'Sex', 'Parch', 'NameLength', 'Child', 'Expensive'], dtype='object')


In [2]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression().fit(x_train, y_train)

print(clf.score(x_train, y_train))
print(clf.score(x_val, y_val))
print(clf.score(x_test,y_test))

0.8127340823970037
0.8333333333333334
0.9688995215311005
